# #The Impact of Clinical Trial Results on Pharmaceutical Stock Performance

While many relate stock market trading to gambling, successful traders have turned stock picking into a science. The likes of Warren Buffet tell us that successful stock buying is all in the research. So what kind of research aids in the prediction of companies within the highly volatile pharmaceutical market? The use of available, open-source APIs and Google Alerts are used to explore if clinical trial results can directly impact stock performance in small, mid, and large cap pharmaceutical companies. Key words and/or phrases in results and related news articles are identified as possible predictors of market effect. As well as a comparison to already established analyst ratings from Barclays, Goldman, Morningstar, or others which have already been shown to impact stock performance.

Step 1) Collect Data

1.1 Set up Google Alerts by going to: https://www.google.com/alerts
        Set alerts to capture news or other articles pertaining to the subject matter
        Collected up to date information on clinical trials using key words: 
            "Phase III trail"
            "Phase 3 trial"
            "Meets Primary End Point"

1.2 Access Google Alert Snippits with Gmail API:
    1.2.1 Go to: https://developers.google.com/gmail/api/quickstart/python to configure Authentication
    1.2.2 Make sure the client_secret.json is in the same directory
    1.2.3 Run Google API parsing script



In [ ]:
'''
Tiffany Fabianac Modified code from:
Reading GMAIL using Python
    - https://github.com/abhishekchhibber/Gmail-Api-through-Python
	- Abhishek Chhibber
'''

'''
This script does the following:
- Go to Gmal inbox
- Find and read all the Google Alert messages
- Extract details (Date, Snippet) and export them to a .csv file / DB
'''

'''
Before running this script, the user should get the authentication by following 
the link: https://developers.google.com/gmail/api/quickstart/python
Also, client_secret.json should be saved in the same directory as this file
'''

# Importing required libraries
from apiclient import discovery
from apiclient import errors
from httplib2 import Http
from oauth2client import file, client, tools
import base64
from bs4 import BeautifulSoup
import re
import time
import dateutil.parser as parser
from datetime import datetime
import datetime
import csv
import json
import io

# Creating a storage.JSON file with authentication details
SCOPES = 'https://www.googleapis.com/auth/gmail.modify'  # we are using modify and not readonly, as we will be marking the messages Read
store = file.Storage('storage.json')
creds = store.get()
if not creds or creds.invalid:
    flow = client.flow_from_clientsecrets('client_secret.json', SCOPES)
    creds = tools.run_flow(flow, store)
GMAIL = discovery.build('gmail', 'v1', http=creds.authorize(Http()))

user_id = 'me'
label_id_one = 'INBOX'

# Getting all the unread messages from Inbox
# labelIds can be changed accordingly
alert_msgs = GMAIL.users().messages().list(userId='me', labelIds=[label_id_one], q='from:googlealerts-noreply@google.com').execute()

# We get a dictonary. Now reading values for the key 'messages'
mssg_list = alert_msgs['messages']

final_list = []

for mssg in mssg_list:
    temp_dict = {}
    m_id = mssg['id']  # get id of individual message
    message = GMAIL.users().messages().get(userId=user_id, id=m_id).execute()  # fetch the message using API
    payld = message['payload']  # get payload of the message
    headr = payld['headers']  # get header of the payload

    for two in headr:  # getting the date
        if two['name'] == 'Date':
            msg_date = two['value']
            date_parse = (parser.parse(msg_date))
            m_date = (date_parse.date())
            temp_dict['Date'] = str(m_date)
        else:
            pass

    temp_dict['Snippet'] = message['snippet']



    final_list=json.dumps(temp_dict)  # This will create a dictonary item in the final list
    re.sub(r'\u22c5', '', final_list)
    print final_list

# exporting the values as .csv
#final_list = json.loads(final_list)
with open("API_out.csv", "a") as f:
    header=["Date", "Snippet"]
    writer = csv.DictWriter(f, fieldnames=header, delimiter=',')
    for x in final_list:
        writer.writerow(x)





1.3 The stock symbols had to be manually extracted from the Snippets (Sad day) 

1.4 Find out if the stocks associate with these Google Alerts had an signigifant (10%) price increase within 5 days (including weekends) from the alert:

In [ ]:
from pandas_datareader import data
import pandas as pd
import csv
import datetime


def stockData (startDate, endDate, ticker):
	# Define which online source one should use
	data_source = 'google'

	# User pandas_reader.data.DataReader to load the desired data.
	panel_data = data.DataReader(ticker, data_source, startDate, endDate)

	close = panel_data.ix['Close']
	volume = panel_data.ix['Volume']
	op = panel_data.ix['Open']
	high = panel_data.ix['High']
	low = panel_data.ix['Low']

	# Getting all weekdays 
	all_weekdays = pd.date_range(start=startDate, end=endDate, freq='B')

	# Align new set of dates
	close = close.reindex(all_weekdays)
	volume = volume.reindex(all_weekdays)
	op = op.reindex(all_weekdays)
	high = high.reindex(all_weekdays)
	low = low.reindex(all_weekdays)

	result = pd.concat([close, volume, op, high, low], axis=1, join='inner')
	result.columns=['close','volume','open','high','low']

	return result
	
    
def findHigh (startDate, ticker):

	# Get date and five days after
	temp_date = datetime.datetime.strptime(startDate, "%Y-%m-%d")
	endDate = temp_date + datetime.timedelta(days=5)

	
	result = stockData(startDate, endDate, ticker)


	tempHigh = result.nlargest(1,'high')
	high = tempHigh.iloc[0]['high']

	return high

def openPrice (startDate, ticker):
	
	result = stockData(startDate, startDate, ticker)
	open = result.iloc[0]['open']
	return open




ticker = ['CLSN']
start_date = '2017-08-21'


with open('google_alert_data.csv', 'rb') as csvfile:
	with open('outfile.csv','a') as f:
		datareader = csv.DictReader(csvfile)
		writer = csv.DictWriter(f, fieldnames=datareader.fieldnames, extrasaction='ignore', 
delimiter=',')
		for line in datareader:
			ticker = [line['Ticker']]
			date = line['\xef\xbb\xbfDate']
			high = findHigh(date, ticker)
			startPrice = openPrice(start_date, ticker)
			prctIncrease = round(((high-startPrice)/startPrice)*100,2)
			if (high > startPrice*1.1):
				line['W/L?']='W'
				line['%Change']=prctIncrease
				print ticker, prctIncrease, high, startPrice
			else:
				line['W/L?']='L'
				line['%Change']=prctIncrease
			writer.writerow(line)


In [15]:
from collections import OrderedDict
from operator import itemgetter
def beautiful_soup(url):
    return "hold"

def text_mine (mySentence):
    mySentence = mySentence.lower()
    myList = [word for word in mySentence.split() if len(word)>3]
    myList = [word for word in myList if word.isalpha()]
    myDict = {}
    for word in myList:
        myDict[word]=[myList.count(word)]

    return myDict

Article_Example = "Celsion Corporation to Post Q4 2017 Earnings of ($0.29) Per Share, Oppenheimer Holdings Forecasts (CLSN) Posted by Paula Ricardo  | Nov 25th, 2017 Celsion Corporation (NASDAQ:CLSN) – Investment analysts at Oppenheimer Holdings issued their Q4 2017 earnings per share (EPS) estimates for Celsion Corporation in a report released on Monday. Oppenheimer Holdings analyst H. Singh anticipates that the biotechnology company will earn ($0.29) per share for the quarter. Oppenheimer Holdings currently has a “Outperform” rating and a $9.00 target price on the stock. Oppenheimer Holdings also issued estimates for Celsion Corporation’s FY2018 earnings at ($1.21) EPS, FY2019 earnings at ($1.32) EPS and FY2020 earnings at ($0.94) EPS. Get Celsion Corporation alerts: A number of other equities analysts also recently issued reports on CLSN. Zacks Investment Research upgraded Celsion Corporation from a “hold” rating to a “buy” rating and set a $1.50 price target for the company in a research note on Tuesday, August 29th. Maxim Group raised shares of Celsion Corporation from a “hold” rating to a “buy” rating and set a $7.00 target price on the stock in a report on Wednesday, October 4th. Finally, ValuEngine cut shares of Celsion Corporation from a “sell” rating to a “strong sell” rating in a report on Wednesday, November 15th. One analyst has rated the stock with a sell rating, one has issued a hold rating and four have issued a buy rating to the company’s stock. Celsion Corporation presently has a consensus rating of “Buy” and a consensus target price of $11.90. Celsion Corporation (NASDAQ CLSN) opened at $3.26 on Thursday. Celsion Corporation has a 12-month low of $1.24 and a 12-month high of $12.46. An institutional investor recently bought a new position in Celsion Corporation stock. Sabby Management LLC purchased a new stake in Celsion Corporation (NASDAQ:CLSN) during the second quarter, according to its most recent filing with the SEC. The fund purchased 108,843 shares of the biotechnology company’s stock, valued at approximately $223,000. Sabby Management LLC owned about 1.30% of Celsion Corporation as of its most recent filing with the SEC. 6.41% of the stock is currently owned by institutional investors. WARNING: “Celsion Corporation to Post Q4 2017 Earnings of ($0.29) Per Share, Oppenheimer Holdings Forecasts (CLSN)” was originally reported by The Lincolnian Online and is the property of of The Lincolnian Online. If you are reading this news story on another domain, it was illegally stolen and republished in violation of United States & international copyright and trademark legislation. The original version of this news story can be viewed at https://www.thelincolnianonline.com/2017/11/25/celsion-corporation-to-post-q4-2017-earnings-of-0-29-per-share-oppenheimer-holdings-forecasts-clsn.html. About Celsion Corporation Celsion Corporation is an oncology drug development company. The Company’s product candidate is ThermoDox, a heat-activated liposomal encapsulation of doxorubicin, which is in Phase III clinical trial for treatment of primary liver cancer (the OPTIMA Study) and a Phase II clinical trial for treatment of recurrent chest wall breast cancer (the DIGNITY Study)."

print OrderedDict(sorted(text_mine(Article_Example).items(), key=itemgetter(1), reverse=True))

OrderedDict([('celsion', [16]), ('corporation', [16]), ('rating', [10]), ('oppenheimer', [6]), ('holdings', [6]), ('earnings', [6]), ('issued', [5]), ('target', [4]), ('price', [4]), ('report', [3]), ('stock', [3]), ('shares', [3]), ('from', [3]), ('with', [3]), ('institutional', [2]), ('treatment', [2]), ('consensus', [2]), ('estimates', [2]), ('investment', [2]), ('currently', [2]), ('research', [2]), ('recently', [2]), ('management', [2]), ('news', [2]), ('about', [2]), ('trial', [2]), ('lincolnian', [2]), ('cancer', [2]), ('sabby', [2]), ('owned', [2]), ('post', [2]), ('story', [2]), ('clinical', [2]), ('company', [2]), ('analyst', [2]), ('this', [2]), ('share', [2]), ('purchased', [2]), ('also', [2]), ('analysts', [2]), ('forecasts', [2]), ('most', [2]), ('biotechnology', [2]), ('phase', [2]), ('recent', [2]), ('filing', [2]), ('oncology', [1]), ('recurrent', [1]), ('four', [1]), ('anticipates', [1]), ('group', [1]), ('copyright', [1]), ('valuengine', [1]), ('ricardo', [1]), ('vio

Step 2) Store Data

Step 3) Analyze Data